In [1]:
import pandas as pd
import time
import datetime
import redis
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

In [2]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="root",  # your password
                     db="pipa_db")        # name of the data base

cur = db.cursor()

In [6]:
# import article data from mysql db 
def import_data_articles(table_name):
    #ds = pd.read_csv(data_source)
    ds = cur.execute('select * from ' +table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['id', 'PMID', 'abstract'])
    

In [4]:
userId =1
userpool = pd.read_csv("./data/testdata.csv")
cur.execute('select pmid, abstract from user_articles where user_id=%d'% (userId))
r = cur.fetchall()
userpool= pd.DataFrame(list(r),columns=['PMID', 'abstract'])
print userpool
print userpool.shape



    PMID                                           abstract
0  10001                  This is a test article from usr1.
1  10002                  This is yet another test article.
2  10003                                                 t3
3  10006  Ultrastructural features of histospecific diff...
(4, 2)


In [8]:
dailypool = import_data_articles('articles')
print dailypool

    id      PMID                                           abstract
0    1  28314358  Effect of Rare-Earth Doping on Free-Volume Nan...
1    2  28314359  Structural Investigation of Uniform Ensembles ...
2    3  28314360  Optimization of the Surface Structure on Black...
3    4  28314361  Hepatotoxicity and Drug/Chemical Interaction T...
4    5  28314362  Design of High Performance Si/SiGe Heterojunct...
5    6  28314363  Raman and Luminescent Spectra of Sulfonated Zn...
6    7  28314364  Effect of Ge Content on the Formation of Ge Na...
7    8  28314365  Cyclic Martensitic Transformations Influence o...
8    9  28314366  A Novel Polar Copolymer Design as a Multi-Func...
9   10  28314367  Nanostructurization of Fe-Ni Alloy.        Abs...
10  11  28314368  Metallic nanoparticles reduce the migration of...
11  12  28314369  Formulation and Drug Loading Features of Nano-...


In [9]:
# refactor the training logic into this function
def train(userpool, dailypool):
    ds = pd.concat([userpool, dailypool])
    ds.reset_index(drop=True, inplace=True)
    
    #clean up not ASCII chars
    ds['abstract'] = ds["abstract"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')
    tfidf_matrix = tf.fit_transform(ds['abstract'])
    print tfidf_matrix.shape
    cosine_similarities = linear_kernel(tfidf_matrix[:userpool.shape[0]], tfidf_matrix[userpool.shape[0]:])
    similarities = pd.DataFrame(cosine_similarities)
    
    prediction_matrix = pd.DataFrame(data=userpool.PMID, columns=['PMID'], index=userpool.index)
    similarity_scores = prediction_matrix.copy()
    
    top_n = 12
    if (ds.shape[0] < top_n):
        top_n = ds.shape[0] + 1

    for i in range(1,top_n-1):
        col_name = i
        prediction_matrix[col_name] = None
        similarity_scores[col_name] = -1

    now = datetime.datetime.now()
    
    for idx in range(userpool.shape[0]):
        print "idx: %s" % idx
        similar_indices = cosine_similarities[idx].argsort()[:-top_n:-1]
        print "similar indices: %s" % similar_indices
        similar_items = [(cosine_similarities[idx][i], ds['PMID'][i]) for i in similar_indices]
        print "similar items: \n %s" % similar_items
        for i in range(1,top_n-1): 

            prediction_matrix.ix[idx, i] = similar_items[i][1] # insert into db directly 
            similarity_scores.ix[idx, i] = similar_items[i][0] # insert into a db table with date

        query = 'insert into recommend_articles values ("%s", %d, %d,\
            %d, %d, %d, %d, %d,\
            %d, %d, %d, %d, %d,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            %s, %s, %s, %s, %s,\
            %s, %s, %s, %s, %s,\)'%(now, userId,userpool.ix[idx,'PMID'],\
                                                         similar_items[0][1],similar_items[1][1],similar_items[2][1],\
                                                         similar_items[3][1],similar_items[4][1],similar_items[5][1],\
                                                         similar_items[6][1],similar_items[7][1],similar_items[8][1],\
                                                         similar_items[9][1],\
                                                         similar_items[0][0],similar_items[1][0],similar_items[2][0],\
                                                         similar_items[3][0],similar_items[4][0],similar_items[5][0],\
                                                         similar_items[6][0],similar_items[7][0],similar_items[8][0],\
                                                         similar_items[9][0])

        cur.execute(query)
        db.commit()
    return 1;

In [10]:
start = time.time()
train(userpool, dailypool)
print("Engine trained in %s seconds." % (time.time() - start))

(16, 3028)
idx: 0
similar indices: [11 10  9  8  7  6  5  4  3  2  1]
similar items: 
 [(0.0, 28314365), (0.0, 28314364), (0.0, 28314363), (0.0, 28314362), (0.0, 28314361), (0.0, 28314360), (0.0, 28314359), (0.0, 28314358), (0.0, 10006), (0.0, 10003), (0.0, 10002)]


OperationalError: (1136, "Column count doesn't match value count at row 1")

In [ ]:
db.close()